# Green Economy Diagnostics (GED) Jupyter Notebook

This Jupyter Notebook performs the end-to-end process to produce the country datacubes as part of the GED's backend. It carries out the following actions 
1. Populates the directory with required sub-directories to store the raw and processed data
2. Takes a valid ISO-3 country code and extracts its shapefiles from the GADM platform
3. Uses the shapefile to extract rasters from various open-sourced platforms / databases to be used as raw data (some steps are still manual because several data sources do not have the necessary APIs)
4. Processes the raw data 
5. Generates the GED's scores 
6. Exports the country's datacube using a specified schema


This notebook is meant to be used using a Colab notebook instance. If used in a local environment, please make sure to install the required libraries specified in the __requirements.txt__ file correctly.

For further questions, please reach out to _sonle.h96@gmail.com_

# Initialization

In [1]:
# If you are using Colab
# !pip install rasterio rasterstats seaborn shapely geopandas rioxarray mapclassify geemap geedim cftime>> /dev/null

import os
import sys
import warnings
warnings.filterwarnings('ignore')
import json

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import rioxarray as rxr
import geopandas as gpd
from osgeo import gdal
import rasterstats
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import seaborn as sns
import xarray as xr

from utils_extract import Extractor
from utils_process import *
from dhelper import get_daterange


In [ ]:
# Mount your google drive to the current Colab instance 
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

# If you are using Colab
dir = '/content/gdrive/MyDrive/wb_pimpam_ged/'
sys.path.append('/content/gdrive/MyDrive/wb_pimpam_ged/')

# Extract

In this section, the country is defined (using its ISO-3 country code), its shapefiles and required raw data are extracted

## Setup

Depending on the country, please make the necessary changes to the _bm_tiles_ subkey under _general_ in the __extract_configs.json__ JSON file. Please contact sonle.h96@gmail.com for the steps required.

A more automated solution is being developed

In [2]:
configs = json.load(open('extract_configs.json', 'r'))

country = 'FRA'

# If you are using Colab
extractor = Extractor(dir, configs)

# If you are using a local environment
# extractor = Extractor('', configs) # replace '' with local directory path

Getting FRA shapefiles for admin 0
Getting FRA shapefiles for admin 1
Getting FRA shapefiles for admin 2


## Pollutants

Here, we extract the raw data of the pollutants used to contruct the Air Quality Score. They are:
* Parcticulate Matter 2.5 (PM 2.5)
* Nitrogen Dioxide (NO2)
* Sulfur Dioxide (SO2)
* Ozone (O3)
* Carbon Monoxide (CO)

### PM 2.5

In [ ]:
extractor.extract_from_gee('pm25', 'mean')

### NO2

In [ ]:
extractor.extract_from_gee('no2', 'mean')

### SO2

In [ ]:
extractor.extract_from_gee('so2', 'mean')

### O3

In [ ]:
extractor.extract_from_gee('pm25', 'mean')

### CO

In [ ]:
extractor.extract_from_gee('co', 'mean')

## Weather

Here, we extract the raw data of temperature and precipitation to used to contruct the Weather / Temperature Score. They are:

* Maximum Temperature 
* Average Temperature 
* Total Precipitation


In [61]:
extractor.extract_from_gee('avg_temp', 'mean')

In [ ]:
extractor.extract_from_gee('max_temp', 'max')

In [ ]:
extractor.extract_from_gee('precipitation', 'sum')

## Land Cover

Here, we extract the raw data of land cover to be used to contruct the Deforestation Score and the Economic Score.

In [ ]:
extractor.extract_land_cover_from_gee()

## Black Marble

Here, we extract the raw data of nighttime luminosity to be used to contruct the Economic Score.

In [ ]:
extractor.extract_bm()

# Process

In this step, we 
1. Calculate the zonal statistics of the raw data
2. Apply further processing to clean the raw data 
3. Use Principal Component Analysis to create the GED's scores

Please input the desired country's ISO-3 code and the desired administrative level in the following code block

In [2]:
country = 'KSV'

# If using Colab
root = f'/content/gdrive/MyDrive/wb_pimpam_ged/{country}/'

# If using the local environment
# root = ''

admin_level = 1
poly = gpd.read_file(os.path.join(root, f'shapefiles/gadm41_{country}_{admin_level}.json'))
poly = poly[[f'GID_{admin_level}', 'geometry']]

## Zonal

### Pollutants

In [88]:
## Pollutants

pm25 = calc_zonal(root, poly, 'pm25', 'mean')[[f'GID_{admin_level}', 'mean', 'date', 'type']]

no2 = calc_zonal(root, poly, 'no2', 'mean')[[f'GID_{admin_level}', 'mean', 'date', 'type']]

so2 = calc_zonal(root, poly, 'so2', 'mean')[[f'GID_{admin_level}', 'mean', 'date', 'type']]

co = calc_zonal(root, poly, 'co', 'mean')[[f'GID_{admin_level}', 'mean', 'date', 'type']]

o3 = calc_zonal(root, poly, 'o3', 'mean')[[f'GID_{admin_level}', 'mean', 'date', 'type']]

df_aqi = pd.concat([pm25, co, no2, so2, o3], axis=0).reset_index(drop=True)
df_aqi['mean'] = df_aqi['mean'].fillna(0)
df_aqi.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/aqi.csv'), index=False)



### Weather

In [3]:
avg_temp = calc_zonal(root, poly, 'avg_temp', 'mean')
avg_temp.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/avg_temp.csv'), index=False)

  0%|          | 0/8401 [00:00<?, ?it/s]

In [4]:
max_temp = calc_zonal(root, poly, 'max_temp', 'max')
max_temp.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/max_temp.csv'), index=False)

  0%|          | 0/8401 [00:00<?, ?it/s]

In [5]:
precipitation = calc_zonal(root, poly, 'precipitation', 'sum')
precipitation.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/precipitation.csv'), index=False)

  0%|          | 0/8401 [00:00<?, ?it/s]

### Land Cover

In [6]:
land_cover = calc_zonal_land_cover(root, poly, 1)
land_cover.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/land_cover.csv'), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

### Black Marble

In [3]:
bm = calc_zonal(root, poly, 'luminosity', 'sum').drop(['date'], axis=1)
bm = bm.groupby(['GID_1', 'year'])[['luminosity']].sum().reset_index()
bm.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/bm.csv'), index=False)

  0%|          | 0/60 [00:00<?, ?it/s]

### Population

No longer is a part of the Economic Score, this section will be removed soon.

In [4]:
pop = calc_zonal_pop(root, poly, [0.992, 0.986], country)
pop.to_csv(os.path.join(root, f'processed_data/admin_{admin_level}/population.csv'), index=False)

2018
2019
2020


## Calculations

In [154]:
country = 'UZB'
root = f'/Users/sonle/Documents/work/WB/GED/{country}/'

admin_level = 2
poly = gpd.read_file(os.path.join(root, f'shapefiles/gadm41_{country}_{admin_level}.json'))
poly = poly[[f'GID_{admin_level}', 'geometry']]

In [155]:
avg_temp = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/avg_temp.csv'))
max_temp = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/max_temp.csv'))
precipitation = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/precipitation.csv'))
aqi = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/aqi.csv'))
land_cover = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/land_cover.csv'))
bm = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/bm.csv'))
pop = pd.read_csv(os.path.join(root, f'processed_data/admin_{admin_level}/population.csv'))



### AQI

In [156]:
df_AQI = calc_aqi(aqi, admin_level)

In [157]:
df_AQI_gb = calc_aqi_agg(df_AQI, admin_level)
df_AQI_gb

,GID_2,YEAR,PM25_SUBINDEX,NO2_SUBINDEX,CO_SUBINDEX,SO2_SUBINDEX,O3_SUBINDEX,PM25_PCT_CHANGE,NO2_PCT_CHANGE,SO2_PCT_CHANGE,CO_PCT_CHANGE,O3_PCT_CHANGE
1,UZB.1.10_1,2019,62.510842,111.875622,1.657144,0.282388,163.985204,-11.017333,-3.308825,-26.893398,291.512418,8.088612
2,UZB.1.10_1,2020,52.592481,106.115188,1.690334,0.651694,162.001012,-15.866624,-5.148963,130.780119,2.002887,-1.209982
3,UZB.1.10_1,2021,41.266028,113.265744,1.715022,0.502349,161.051694,-21.536260,6.738485,-22.916456,1.460528,-0.585995
4,UZB.1.10_1,2022,32.748866,113.977272,1.579503,0.459617,158.071765,-20.639645,0.628194,-8.506473,-7.901886,-1.850293
6,UZB.1.11_1,2019,49.733924,109.416287,1.601175,0.353400,163.301572,-5.869680,-5.081862,8.105632,304.663648,8.210866
...,...,...,...,...,...,...,...,...,...,...,...,...
799,UZB.9.7_1,2022,30.791057,76.636303,1.525999,0.318138,162.197068,-7.874347,-5.428348,-17.483366,-10.492752,-1.313166
801,UZB.9.8_1,2019,102.682921,98.905214,1.566233,0.451545,162.595024,-26.449543,-3.048078,-36.128967,290.865113,8.332347
802,UZB.9.8_1,2020,50.910262,94.770278,1.583656,0.859779,160.833197,-50.419932,-4.180707,90.408269,1.112438,-1.083567
803,UZB.9.8_1,2021,37.289068,105.852607,1.610007,0.624225,159.795980,-26.755301,11.693887,-27.397041,1.663965,-0.644902


In [158]:
df_AQI_bucket = calc_AQI_buckets(df_AQI, admin_level)
df_AQI_bucket

AQI_bucket,GID_2,year,Moderate,Poor,Severe,Very Poor,Good,Satisfactory
0,UZB.1.10_1,2019,89.589041,8.767123,0.273973,1.369863,0.0,0.0
1,UZB.1.10_1,2020,94.535519,5.191257,0.000000,0.273224,0.0,0.0
2,UZB.1.10_1,2021,93.150685,6.849315,0.000000,0.000000,0.0,0.0
3,UZB.1.10_1,2022,93.698630,5.753425,0.000000,0.547945,0.0,0.0
4,UZB.1.11_1,2019,91.780822,7.945205,0.000000,0.273973,0.0,0.0
...,...,...,...,...,...,...,...,...
639,UZB.9.7_1,2022,96.986301,3.013699,0.000000,0.000000,0.0,0.0
640,UZB.9.8_1,2019,82.191781,12.602740,1.095890,4.109589,0.0,0.0
641,UZB.9.8_1,2020,94.808743,5.191257,0.000000,0.000000,0.0,0.0
642,UZB.9.8_1,2021,95.890411,4.109589,0.000000,0.000000,0.0,0.0


### Weather

In [159]:
# Extreme temperatures 
avg_temp = avg_temp.rename({'avg_temp': 'average_temperature'}, axis=1)
temp_percentiles = calc_temp_percentiles(avg_temp, 
                                         start_date='2000-01-01', 
                                         end_date='2010-01-01', 
                                         admin_level=admin_level)

df_extreme_temp = calc_extreme_temp(avg_temp, temp_percentiles, 
                                    start_date='2018-01-01', 
                                    end_date='2022-12-31',
                                    admin_level=admin_level)

df_extreme_temp



,GID_2,year,extremely_hot,extremely_cold,extremely_hot_pct_change,extremely_cold_pct_change
1,UZB.1.10_1,2019,0.115068,0.049315,13.513514,-35.714286
2,UZB.1.10_1,2020,0.098361,0.150273,-14.519906,204.720704
3,UZB.1.10_1,2021,0.000000,0.249315,-100.000000,65.907846
4,UZB.1.10_1,2022,0.000000,0.422222,NaN,69.352869
6,UZB.1.11_1,2019,0.109589,0.035616,8.108108,0.000000
...,...,...,...,...,...,...
799,UZB.9.7_1,2022,0.000000,0.333333,-100.000000,182.945736
801,UZB.9.8_1,2019,0.109589,0.043836,-13.043478,-44.827586
802,UZB.9.8_1,2020,0.117486,0.155738,7.206284,255.276639
803,UZB.9.8_1,2021,0.005479,0.126027,-95.336094,-19.077145


In [160]:
# Max temperature 
df_max_temp = calc_max_temp(max_temp, 
                            start_date='2018-01-01', 
                            end_date='2022-12-31',
                            admin_level=admin_level)
df_max_temp

,GID_2,year,max_temp,MAX_TEMP_PCT_CHANGE
1,UZB.1.10_1,2019,40.687782,-0.395223
2,UZB.1.10_1,2020,39.416428,-3.124660
3,UZB.1.10_1,2021,42.722961,8.388720
4,UZB.1.10_1,2022,20.451920,-52.128975
6,UZB.1.11_1,2019,38.004055,-0.475975
...,...,...,...,...
799,UZB.9.7_1,2022,20.915203,-54.862588
801,UZB.9.8_1,2019,40.429466,-2.151047
802,UZB.9.8_1,2020,38.213280,-5.481612
803,UZB.9.8_1,2021,41.759914,9.281157


In [161]:
prec_percentiles = calc_prec_percentiles(precipitation,
                                         start_date='2000-01-01',
                                         end_date='2010-12-31',
                                         admin_level=admin_level)

df_extreme_prec = calc_extreme_prec(precipitation,
                                    prec_percentiles,
                                    start_date='2018-01-01',
                                    end_date='2022-12-31',
                                    admin_level=admin_level)

df_extreme_prec

,GID_2,year,extremely_wet,extremely_dry,extremely_wet_pct_change,extremely_dry_pct_change
0,UZB.1.10_1,2018,0.101370,0.120548,NaN,NaN
1,UZB.1.10_1,2019,0.101370,0.000000,0.000000,-100.000000
2,UZB.1.10_1,2020,0.079235,0.090164,-21.835770,100.000000
3,UZB.1.10_1,2021,0.079452,0.076712,0.273973,-14.919054
4,UZB.1.10_1,2022,0.118644,0.000000,49.327878,-100.000000
...,...,...,...,...,...,...
800,UZB.9.8_1,2018,0.065753,0.191781,NaN,NaN
801,UZB.9.8_1,2019,0.120548,0.000000,83.333333,-100.000000
802,UZB.9.8_1,2020,0.120219,0.051913,-0.273224,100.000000
803,UZB.9.8_1,2021,0.049315,0.112329,-58.978829,116.380678


In [162]:
df_prec_max = calc_max_prec(precipitation,
                            start_date='2018-01-01',
                            end_date='2022-12-31',
                            admin_level=admin_level)
df_prec_max

,year,GID_2,precipitation_max,precipitation_max_pct_change
1,2019,UZB.1.10_1,6769.077148,-10.846464
2,2020,UZB.1.10_1,6147.477051,-9.182937
3,2021,UZB.1.10_1,7036.174316,14.456293
4,2022,UZB.1.10_1,3034.204590,-56.877069
6,2019,UZB.1.11_1,16020.100586,-30.038700
...,...,...,...,...
799,2022,UZB.9.7_1,762496.062500,-9.053075
801,2019,UZB.9.8_1,52585.160156,-2.680054
802,2020,UZB.9.8_1,52424.273438,-0.305955
803,2021,UZB.9.8_1,37820.636719,-27.856632


### Luminosity & Population

In [182]:
bm = bm.rename({'LUMINOSITY_SUM': 'luminosity'}, axis=1)

In [183]:
bm

,GID_2,year,luminosity
0,UZB.1.10_1,2018,27832.10
1,UZB.1.10_1,2019,29980.75
2,UZB.1.10_1,2020,29995.95
3,UZB.1.10_1,2021,30593.90
4,UZB.1.10_1,2022,35506.40
...,...,...,...
800,UZB.9.8_1,2018,50435.15
801,UZB.9.8_1,2019,40733.10
802,UZB.9.8_1,2020,36853.00
803,UZB.9.8_1,2021,44468.85


In [ ]:
bm.columns = bm.columns.str.upper()
bm['LUMINOSITY_PCT_CHANGE'] = 100*(bm.groupby('GID_2')['LUMINOSITY'].apply(pd.Series.pct_change))
bm.head()

In [186]:
# df_econ = calc_lpc(bm, pop,
#                    start_year=2018,
#                    end_year=2022,
#                    admin_level=admin_level)
# df_econ.columns = df_econ.columns.str.upper()
# df_econ

,GID_2,YEAR,LUMINOSITY,POPULATION,LPC,LPC_PCT_CHANGE
1,UZB.1.10_1,2019,29980.75,206130.000000,0.145445,6.632553
2,UZB.1.10_1,2020,29995.95,221590.000000,0.135366,-6.929656
3,UZB.1.10_1,2021,30593.90,225246.235000,0.135824,0.337868
4,UZB.1.10_1,2022,35506.40,228850.174760,0.155151,14.229459
6,UZB.1.11_1,2019,51708.05,363064.000000,0.142421,-3.112312
...,...,...,...,...,...,...
799,UZB.9.7_1,2022,21260.80,102465.680260,0.207490,11.756660
801,UZB.9.8_1,2019,40733.10,375291.000000,0.108537,-22.154382
802,UZB.9.8_1,2020,36853.00,399357.000000,0.092281,-14.977807
803,UZB.9.8_1,2021,44468.85,405946.390500,0.109543,18.706824


# Compute Scores

In [165]:
for d in [df_AQI_bucket, df_extreme_temp, df_max_temp, df_prec_max, df_extreme_prec, land_cover]:
    d.columns = d.columns.str.upper()

## Environment Score

In [166]:
df = pd.merge(df_AQI_bucket, df_AQI_gb, on=[f'GID_{admin_level}', 'YEAR'], how='left')
df = pd.merge(df, df_extreme_temp, on=[f'GID_{admin_level}', 'YEAR'], how='left')
df = pd.merge(df, df_max_temp, on=[f'GID_{admin_level}', 'YEAR'], how='left')
df = pd.merge(df, df_prec_max, on=[f'GID_{admin_level}', 'YEAR'], how='left')
df = pd.merge(df, df_extreme_prec, on=[f'GID_{admin_level}', 'YEAR'], how='left')
df = pd.merge(df, land_cover , on=[f'GID_{admin_level}', 'YEAR'], how='left')

df = df.fillna(0)
df = df[df['YEAR'].between(2019, 2022)].reset_index(drop=True)
df = df.replace([np.inf], 100)

df_o = df.copy(deep=True)

In [167]:
df = df.drop(['PM25_SUBINDEX', 'NO2_SUBINDEX', 'SO2_SUBINDEX', 'CO_SUBINDEX', 'O3_SUBINDEX', 
              'MAX_TEMP', 'PRECIPITATION_MAX'], axis=1)

negative_vars = ['POOR', 'VERY POOR', 'SEVERE', 'PM25_PCT_CHANGE', 'NO2_PCT_CHANGE',
                'SO2_PCT_CHANGE', 'CO_PCT_CHANGE', 'O3_PCT_CHANGE', 'EXTREMELY_HOT',
                'EXTREMELY_COLD', 'EXTREMELY_HOT_PCT_CHANGE',
                'EXTREMELY_COLD_PCT_CHANGE', 'MAX_TEMP_PCT_CHANGE',
                'PRECIPITATION_MAX_PCT_CHANGE', 'EXTREMELY_WET', 'EXTREMELY_DRY',
                'EXTREMELY_WET_PCT_CHANGE', 'EXTREMELY_DRY_PCT_CHANGE']

for var in negative_vars:
    df[var] = df[var]*-1

# built cover +ive growth only affects the environmental score negatively when green cover growth is -ive 
df['BUILT_COVER_GROWTH'] = np.where(df['GREEN_COVER_GROWTH'] < 0, 
                                    df['BUILT_COVER_GROWTH']*-1,
                                    0)


# Extract the environmental variables you want to include in the index
air_vars = ['GOOD', 'SATISFACTORY', 'MODERATE', 'POOR', 'VERY POOR', 'SEVERE',
            'PM25_PCT_CHANGE', 'NO2_PCT_CHANGE', 'SO2_PCT_CHANGE',
            'CO_PCT_CHANGE', 'O3_PCT_CHANGE']
temp_vars = ['EXTREMELY_HOT', 'EXTREMELY_COLD',
             'EXTREMELY_HOT_PCT_CHANGE', 'EXTREMELY_COLD_PCT_CHANGE',
             'MAX_TEMP_PCT_CHANGE', 'PRECIPITATION_MAX_PCT_CHANGE', 'EXTREMELY_WET',
             'EXTREMELY_DRY', 'EXTREMELY_WET_PCT_CHANGE', 'EXTREMELY_DRY_PCT_CHANGE']
forest_vars = ['GREEN_PCT', 'GREEN_COVER_GROWTH', 'BUILT_COVER_GROWTH']


df_o['AIR_SCORE'] = get_pca_comp(df, air_vars)
df_o['AIR_SCORE'] = df_o.groupby(['YEAR'])['AIR_SCORE'].rank(pct=True, ascending=True)*100
df_o['TEMP_SCORE'] = get_pca_comp(df, temp_vars)
df_o['TEMP_SCORE'] = df_o.groupby(['YEAR'])['TEMP_SCORE'].rank(pct=True, ascending=True)*100
df_o['FOREST_SCORE'] = get_pca_comp(df, forest_vars)
df_o['FOREST_SCORE'] = df_o.groupby(['YEAR'])['FOREST_SCORE'].rank(pct=True, ascending=True)*100

df_o['ENVR_SCORE'] = np.round((df_o['AIR_SCORE']+df_o['TEMP_SCORE']+df_o['FOREST_SCORE'])/3, 2)

df_o

,GID_2,YEAR,MODERATE,POOR,SEVERE,VERY POOR,GOOD,SATISFACTORY,PM25_SUBINDEX,NO2_SUBINDEX,CO_SUBINDEX,SO2_SUBINDEX,O3_SUBINDEX,PM25_PCT_CHANGE,NO2_PCT_CHANGE,SO2_PCT_CHANGE,CO_PCT_CHANGE,O3_PCT_CHANGE,EXTREMELY_HOT,EXTREMELY_COLD,EXTREMELY_HOT_PCT_CHANGE,EXTREMELY_COLD_PCT_CHANGE,MAX_TEMP,MAX_TEMP_PCT_CHANGE,PRECIPITATION_MAX,PRECIPITATION_MAX_PCT_CHANGE,EXTREMELY_WET,EXTREMELY_DRY,EXTREMELY_WET_PCT_CHANGE,EXTREMELY_DRY_PCT_CHANGE,GREEN_PCT,BUILT_PCT,GREEN_COVER_GROWTH,BUILT_COVER_GROWTH,AIR_SCORE,TEMP_SCORE,FOREST_SCORE,ENVR_SCORE
0,UZB.1.10_1,2019,89.589041,8.767123,0.273973,1.369863,0.0,0.0,62.510842,111.875622,1.657144,0.282388,163.985204,-11.017333,-3.308825,-26.893398,291.512418,8.088612,0.115068,0.049315,13.513514,-35.714286,40.687782,-0.395223,6769.077148,-10.846464,0.101370,0.000000,0.000000,-100.000000,0.000000,24.738676,0.000000,5.341246,14.906832,26.086957,30.434783,23.81
1,UZB.1.10_1,2020,94.535519,5.191257,0.000000,0.273224,0.0,0.0,52.592481,106.115188,1.690334,0.651694,162.001012,-15.866624,-5.148963,130.780119,2.002887,-1.209982,0.098361,0.150273,-14.519906,204.720704,39.416428,-3.124660,6147.477051,-9.182937,0.079235,0.090164,-21.835770,100.000000,0.000000,25.017422,0.000000,1.126761,44.720497,63.354037,67.080745,58.39
2,UZB.1.10_1,2021,93.150685,6.849315,0.000000,0.000000,0.0,0.0,41.266028,113.265744,1.715022,0.502349,161.051694,-21.536260,6.738485,-22.916456,1.460528,-0.585995,0.000000,0.249315,-100.000000,65.907846,42.722961,8.388720,7036.174316,14.456293,0.079452,0.076712,0.273973,-14.919054,0.000000,24.041812,0.000000,-3.899721,40.993789,96.273292,59.627329,65.63
3,UZB.1.10_1,2022,93.698630,5.753425,0.000000,0.547945,0.0,0.0,32.748866,113.977272,1.579503,0.459617,158.071765,-20.639645,0.628194,-8.506473,-7.901886,-1.850293,0.000000,0.422222,0.000000,69.352869,20.451920,-52.128975,3034.204590,-56.877069,0.118644,0.000000,49.327878,-100.000000,0.000000,25.435540,0.000000,5.797101,65.838509,14.285714,28.260870,36.13
4,UZB.1.11_1,2019,91.780822,7.945205,0.000000,0.273973,0.0,0.0,49.733924,109.416287,1.601175,0.353400,163.301572,-5.869680,-5.081862,8.105632,304.663648,8.210866,0.109589,0.035616,8.108108,0.000000,38.004055,-0.475975,16020.100586,-30.038700,0.090411,0.000000,-13.157895,-100.000000,1.311953,18.613550,-38.983051,1.823908,7.453416,35.403727,9.316770,17.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,UZB.9.7_1,2022,96.986301,3.013699,0.000000,0.000000,0.0,0.0,30.791057,76.636303,1.525999,0.318138,162.197068,-7.874347,-5.428348,-17.483366,-10.492752,-1.313166,0.000000,0.333333,-100.000000,182.945736,20.915203,-54.862588,762496.062500,-9.053075,0.152542,0.000000,73.993644,-100.000000,0.002824,0.012505,-93.859649,19.230790,42.236025,46.583851,1.242236,30.02
640,UZB.9.8_1,2019,82.191781,12.602740,1.095890,4.109589,0.0,0.0,102.682921,98.905214,1.566233,0.451545,162.595024,-26.449543,-3.048078,-36.128967,290.865113,8.332347,0.109589,0.043836,-13.043478,-44.827586,40.429466,-2.151047,52585.160156,-2.680054,0.120548,0.000000,83.333333,-100.000000,0.921252,17.956014,135.483871,-0.076908,56.521739,83.229814,79.503106,73.08
641,UZB.9.8_1,2020,94.808743,5.191257,0.000000,0.000000,0.0,0.0,50.910262,94.770278,1.583656,0.859779,160.833197,-50.419932,-4.180707,90.408269,1.112438,-1.083567,0.117486,0.155738,7.206284,255.276639,38.213280,-5.481612,52424.273438,-0.305955,0.120219,0.051913,-0.273224,100.000000,0.757193,18.271853,-17.808219,1.758958,32.298137,93.167702,34.161491,53.21
642,UZB.9.8_1,2021,95.890411,4.109589,0.000000,0.000000,0.0,0.0,37.289068,105.852607,1.610007,0.624225,159.795980,-26.755301,11.693887,-27.397041,1.663965,-0.644902,0.005479,0.126027,-95.336094,-19.077145,41.759914,9.281157,37820.636719,-27.856632,0.049315,0.112329,-58.978829,116.380678,0.403836,18.663865,-46.666667,2.145439,15.527950,36.645963,7.453416,19.88


## Economic Score

In [187]:
# df_econ = pd.merge(df_econ, land_cover.drop(['GREEN_PCT', 'GREEN_COVER_GROWTH'], axis=1),
#                    on=[f'GID_{admin_level}', 'YEAR'], how='left')

df_econ = pd.merge(bm, land_cover.drop(['GREEN_PCT', 'GREEN_COVER_GROWTH'], axis=1),
                   on=[f'GID_{admin_level}', 'YEAR'], how='left')

df_econ['BUILT_COVER_GROWTH'] = df_econ['BUILT_COVER_GROWTH'].replace(np.inf, 0)
df_econ = df_econ.fillna(0)

# econ_vars = ['LPC', 'LPC_PCT_CHANGE', 'BUILT_PCT', 'BUILT_COVER_GROWTH']
econ_vars = ['LUMINOSITY', 'LUMINOSITY_PCT_CHANGE']

df_econ['ECON_SCORE'] = get_pca_comp(df_econ, econ_vars)
df_econ = df_econ.sort_values(by=[f'GID_{admin_level}', 'YEAR']).reset_index(drop=True)
df_econ['ECON_SCORE'] = df_econ.groupby(['YEAR'])['ECON_SCORE'].rank(pct=True, ascending=True)*100

# Export & Sanity Checks

## Compiling

In [188]:
gdf = gpd.read_file(os.path.join(root, f'shapefiles/gadm41_{country}_{admin_level}.json'))


In [196]:
if admin_level == 1:
    gdf = gdf[['GID_1', 'NAME_1']]
    cols_keep = ['PROVINCE_ID', 'PROVINCE', 'YEAR']

if admin_level == 2:
    gdf = gdf[['GID_1', 'NAME_1', 'GID_2', 'NAME_2']]
    cols_keep = ['PROVINCE_ID', 'PROVINCE', 'DISTRICT_ID', 'DISTRICT', 'YEAR']

In [190]:
df_full = pd.merge(df_econ, df_o.drop(['BUILT_PCT', 'BUILT_COVER_GROWTH'], axis=1), 
                   on=[f'GID_{admin_level}', 'YEAR'], how='left')

df_full = pd.merge(gdf, df_full, on=[f'GID_{admin_level}'], how='right')

z_cols = ['LPC', 'LPC_PCT_CHANGE', 'PM25_SUBINDEX', 'NO2_SUBINDEX', 'CO_SUBINDEX', 'SO2_SUBINDEX',
          'O3_SUBINDEX','EXTREMELY_HOT', 'EXTREMELY_COLD', 'MAX_TEMP', 'PRECIPITATION_MAX', 'EXTREMELY_WET', 'EXTREMELY_DRY',
          'GREEN_PCT', 'GREEN_COVER_GROWTH', 'BUILT_PCT', 'BUILT_COVER_GROWTH']

df_full = calc_z_scores(df_full, z_cols)



In [191]:
for col in ['LPC_STD', 'LPC_PCT_CHANGE_STD', 'BUILT_PCT_STD', 'BUILT_COVER_GROWTH_STD']:
    df_full = df_full.rename({col: 'ECON_' + col}, axis=1)

for col in ['PM25_SUBINDEX_STD', 'NO2_SUBINDEX_STD', 'CO_SUBINDEX_STD', 'SO2_SUBINDEX_STD', 'O3_SUBINDEX_STD']:
    df_full = df_full.rename({col: 'AIR_' + col}, axis=1)

for col in ['EXTREMELY_HOT_STD', 'EXTREMELY_COLD_STD', 'MAX_TEMP_STD', 'PRECIPITATION_MAX_STD', 'EXTREMELY_WET_STD', 'EXTREMELY_DRY_STD']:
    df_full = df_full.rename({col: 'TEMP_' + col}, axis=1)

for col in ['GREEN_PCT_STD', 'GREEN_COVER_GROWTH_STD']:
    df_full = df_full.rename({col: 'FOREST_' + col}, axis=1)


In [192]:
df_full.head()

,GID_1,NAME_1,GID_2,NAME_2,YEAR,LUMINOSITY,POPULATION,LPC,LPC_PCT_CHANGE,BUILT_PCT,BUILT_COVER_GROWTH,ECON_SCORE,MODERATE,POOR,SEVERE,VERY POOR,GOOD,SATISFACTORY,PM25_SUBINDEX,NO2_SUBINDEX,CO_SUBINDEX,SO2_SUBINDEX,O3_SUBINDEX,PM25_PCT_CHANGE,NO2_PCT_CHANGE,SO2_PCT_CHANGE,CO_PCT_CHANGE,O3_PCT_CHANGE,EXTREMELY_HOT,EXTREMELY_COLD,EXTREMELY_HOT_PCT_CHANGE,EXTREMELY_COLD_PCT_CHANGE,MAX_TEMP,MAX_TEMP_PCT_CHANGE,PRECIPITATION_MAX,PRECIPITATION_MAX_PCT_CHANGE,EXTREMELY_WET,EXTREMELY_DRY,EXTREMELY_WET_PCT_CHANGE,EXTREMELY_DRY_PCT_CHANGE,GREEN_PCT,GREEN_COVER_GROWTH,AIR_SCORE,TEMP_SCORE,FOREST_SCORE,ENVR_SCORE,ECON_LPC_STD,ECON_LPC_PCT_CHANGE_STD,AIR_PM25_SUBINDEX_STD,AIR_NO2_SUBINDEX_STD,AIR_CO_SUBINDEX_STD,AIR_SO2_SUBINDEX_STD,AIR_O3_SUBINDEX_STD,TEMP_EXTREMELY_HOT_STD,TEMP_EXTREMELY_COLD_STD,TEMP_MAX_TEMP_STD,TEMP_PRECIPITATION_MAX_STD,TEMP_EXTREMELY_WET_STD,TEMP_EXTREMELY_DRY_STD,FOREST_GREEN_PCT_STD,FOREST_GREEN_COVER_GROWTH_STD,ECON_BUILT_PCT_STD,ECON_BUILT_COVER_GROWTH_STD
0,UZB.1_1,Andijon,UZB.1.10_1,Paxtaobod,2019,29980.75,206130.00000,0.145445,6.632553,24.738676,5.341246,34.782609,89.589041,8.767123,0.273973,1.369863,0.0,0.0,62.510842,111.875622,1.657144,0.282388,163.985204,-11.017333,-3.308825,-26.893398,291.512418,8.088612,0.115068,0.049315,13.513514,-35.714286,40.687782,-0.395223,6769.077148,-10.846464,0.101370,0.000000,0.000000,-100.000000,0.000000,0.000000,14.906832,26.086957,30.434783,23.81,-0.076850,-0.199932,0.129745,-0.024212,0.449309,-1.190624,0.960164,0.903457,-1.044086,0.476918,-0.377237,-0.033996,-0.793846,-0.481795,-0.196752,0.744046,0.108369
1,UZB.1_1,Andijon,UZB.1.10_1,Paxtaobod,2020,29995.95,221590.00000,0.135366,-6.929656,25.017422,1.126761,17.391304,94.535519,5.191257,0.000000,0.273224,0.0,0.0,52.592481,106.115188,1.690334,0.651694,162.001012,-15.866624,-5.148963,130.780119,2.002887,-1.209982,0.098361,0.150273,-14.519906,204.720704,39.416428,-3.124660,6147.477051,-9.182937,0.079235,0.090164,-21.835770,100.000000,0.000000,0.000000,44.720497,63.354037,67.080745,58.39,-0.076863,-0.618295,-0.158377,-0.204016,0.709292,0.386722,0.387292,0.623220,-0.193665,0.340850,-0.381637,-0.599127,1.294551,-0.481795,-0.196752,0.765090,-0.133671
2,UZB.1_1,Andijon,UZB.1.10_1,Paxtaobod,2021,30593.90,225246.23500,0.135824,0.337868,24.041812,-3.899721,9.937888,93.150685,6.849315,0.000000,0.000000,0.0,0.0,41.266028,113.265744,1.715022,0.502349,161.051694,-21.536260,6.738485,-22.916456,1.460528,-0.585995,0.000000,0.249315,-100.000000,65.907846,42.722961,8.388720,7036.174316,14.456293,0.079452,0.076712,0.273973,-14.919054,0.000000,0.000000,40.993789,96.273292,59.627329,65.63,-0.076862,-0.394109,-0.487403,0.019179,0.902672,-0.251146,0.113206,-1.026565,0.640614,0.694736,-0.375346,-0.593585,0.982982,-0.481795,-0.196752,0.691436,-0.422343
3,UZB.1_1,Andijon,UZB.1.10_1,Paxtaobod,2022,35506.40,228850.17476,0.155151,14.229459,25.435540,5.797101,27.950311,93.698630,5.753425,0.000000,0.547945,0.0,0.0,32.748866,113.977272,1.579503,0.459617,158.071765,-20.639645,0.628194,-8.506473,-7.901886,-1.850293,0.000000,0.422222,0.000000,69.352869,20.451920,-52.128975,3034.204590,-56.877069,0.118644,0.000000,49.327878,-100.000000,0.000000,0.000000,65.838509,14.285714,28.260870,36.13,-0.076837,0.034414,-0.734821,0.041388,-0.158851,-0.433660,-0.747153,-1.026565,2.097098,-1.688851,-0.403679,0.407037,-0.793846,-0.481795,-0.196752,0.796656,0.134549
4,UZB.1_1,Andijon,UZB.1.11_1,Qo'rg'ontepa,2019,51708.05,363064.00000,0.142421,-3.112312,18.613550,1.823908,26.086957,91.780822,7.945205,0.000000,0.273973,0.0,0.0,49.733924,109.416287,1.601175,0.353400,163.301572,-5.869680,-5.081862,8.105632,304.663648,8.210866,0.109589,0.035616,8.108108,0.000000,38.004055,-0.475975,16020.100586,-30.038700,0.090411,0.000000,-13.157895,-100.000000,1.311953,-38.983051,7.453416,35.403727,9.316770,17.39,-0.076854,-0.500539,-0.241416,-0.100976,0.010909,-0.887325,0.762787,0.811552,-1.159476,0.189689,-0.311740,-0.313790,-0.793846,-0.347242,-0.391538,0.281624,-0.093633


In [193]:
df_full = df_full.rename({'GID_1': 'PROVINCE_ID',
                          'NAME_1': 'PROVINCE',
                          'GID_2': 'DISTRICT_ID',
                          'NAME_2': 'DISTRICT'}, axis=1)

df_full.to_csv(os.path.join(root, f'datasets/WB_GPBP_GED_{country}_{admin_level}_full.csv'), index=False)


In [197]:
cols_keep.append('ECON_SCORE')

In [198]:
# cols_keep.extend(df_full.columns[40:])
cols_keep.extend(df_full.columns[42:])
cols_keep

['PROVINCE_ID',
 'PROVINCE',
 'DISTRICT_ID',
 'DISTRICT',
 'YEAR',
 'ECON_SCORE',
 'AIR_SCORE',
 'TEMP_SCORE',
 'FOREST_SCORE',
 'ENVR_SCORE',
 'ECON_LPC_STD',
 'ECON_LPC_PCT_CHANGE_STD',
 'AIR_PM25_SUBINDEX_STD',
 'AIR_NO2_SUBINDEX_STD',
 'AIR_CO_SUBINDEX_STD',
 'AIR_SO2_SUBINDEX_STD',
 'AIR_O3_SUBINDEX_STD',
 'TEMP_EXTREMELY_HOT_STD',
 'TEMP_EXTREMELY_COLD_STD',
 'TEMP_MAX_TEMP_STD',
 'TEMP_PRECIPITATION_MAX_STD',
 'TEMP_EXTREMELY_WET_STD',
 'TEMP_EXTREMELY_DRY_STD',
 'FOREST_GREEN_PCT_STD',
 'FOREST_GREEN_COVER_GROWTH_STD',
 'ECON_BUILT_PCT_STD',
 'ECON_BUILT_COVER_GROWTH_STD']

In [199]:
df_reduced = df_full[cols_keep]
df_reduced.to_csv(os.path.join(root, f'datasets/WB_GPBP_GED_{country}_{admin_level}_scores.csv'), index=False)


In [200]:
percent_missing = df_reduced.isnull().sum() * 100 / len(df_reduced)
missing_value_df = pd.DataFrame({'column_name': df_reduced.columns,
                                 'percent_missing': percent_missing})

In [201]:
missing_value_df

,column_name,percent_missing
PROVINCE_ID,PROVINCE_ID,0.0
PROVINCE,PROVINCE,0.0
DISTRICT_ID,DISTRICT_ID,0.0
DISTRICT,DISTRICT,0.0
YEAR,YEAR,0.0
ECON_SCORE,ECON_SCORE,0.0
AIR_SCORE,AIR_SCORE,0.0
TEMP_SCORE,TEMP_SCORE,0.0
FOREST_SCORE,FOREST_SCORE,0.0
ENVR_SCORE,ENVR_SCORE,0.0
